In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_gnn as tfgnn
import torch as th
from torch import nn as nn
import torch_geometric as pyg
import dgl
import numpy as np

2022-07-18 16:20:23.637846: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-18 16:20:23.637876: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[16:20:30] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /home/mparadkar/miniconda3/lib/python3.9/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.12.0.so: cannot open shared object file: No such file or directory
Using backend: pytorch


In [2]:
x_dict = {
    "airframe": th.eye(4, 3),
    "powerplant": th.eye(3, 2),
}
edge_index_dict = {
    ("airplane", "powered by", "powerplant"): th.tensor(
        [[0, 1, 2, 3],
         [0, 1, 2, 1]]
    )
}

In [3]:
from torch_geometric.data import HeteroData

data = HeteroData()
data['airframe'].x = th.eye(4, 3)
data['powerplant'].x = th.eye(3, 2)
data['airframe', 'powered by', 'powerplant'].edge_index = th.tensor(
        [[0, 1, 2, 3],
         [0, 1, 2, 1]]
    )
data['airframe', 'developed from', 'airframe'].edge_index = th.tensor(
        [[1, 3],
         [0, 1]]
)

In [4]:
import torch_geometric.transforms as T
T.AddSelfLoops()(data)['airframe', 'developed from', 'airframe']

{'edge_index': tensor([[1, 3, 0, 1, 2, 3],
        [0, 1, 0, 1, 2, 3]])}

In [5]:
T.AddSelfLoops()(data)['airframe', 'powered by', 'powerplant']

{'edge_index': tensor([[0, 1, 2, 3],
        [0, 1, 2, 1]])}

In [6]:
data.edge_stores[1].is_bipartite()

False

In [7]:
channels_dict = {node_type: data[node_type].num_features for node_type in data.node_types}

In [8]:
data['airframe'].num_features

3

In [9]:
hgtc_pyg = pyg.nn.HGTConv(in_channels=channels_dict, out_channels=2, metadata=data.metadata(), heads=2)

In [10]:
hgtc_pyg.k_lin

ModuleDict(
  (airframe): Linear(3, 2, bias=True)
  (powerplant): Linear(2, 2, bias=True)
)

In [11]:
hgtc_pyg.p_rel['airframe__powered by__powerplant']

Parameter containing:
tensor([1., 1.], requires_grad=True)

In [12]:
hgtc_pyg(data.x_dict, data.edge_index_dict)

{'airframe': tensor([[ 0.3397, -0.5333],
         [ 0.2330, -0.6602],
         [ 0.3285, -0.5557],
         [ 0.2179, -0.6497]], grad_fn=<AddmmBackward0>),
 'powerplant': tensor([[0.5096, 0.3893],
         [0.0470, 0.6210],
         [0.1575, 0.3758]], grad_fn=<AddBackward0>)}

In [26]:
import math
from typing import Dict, List, Optional, Union, Any

import torch
import torch.nn.functional as F
from torch import Tensor
from torch.nn import init, Parameter
from torch_sparse import SparseTensor

from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense import Linear
from torch_geometric.nn.inits import glorot, ones, reset
from torch_geometric.typing import EdgeType, Metadata, NodeType
from torch_geometric.utils import softmax

def eye(value: Any):
    if isinstance(value, Tensor):
        init.eye_(value)
    else:
        for v in value.parameters() if hasattr(value, 'parameters') else []:
            eye(v)
        for v in value.buffers() if hasattr(value, 'buffers') else []:
            eye(v)
            
def group(xs: List[Tensor], aggr: Optional[str]) -> Optional[Tensor]:
    if len(xs) == 0:
        return None
    elif aggr is None:
        return torch.stack(xs, dim=1)
    elif len(xs) == 1:
        return xs[0]
    else:
        out = torch.stack(xs, dim=0)
        out = getattr(torch, aggr)(out, dim=0)
        out = out[0] if isinstance(out, tuple) else out
        return out


class HackedGTConv(MessagePassing):
    r"""The Heterogeneous Graph Transformer (HGT) operator from the
    `"Heterogeneous Graph Transformer" <https://arxiv.org/abs/2003.01332>`_
    paper.

    .. note::

        For an example of using HGT, see `examples/hetero/hgt_dblp.py
        <https://github.com/pyg-team/pytorch_geometric/blob/master/examples/
        hetero/hgt_dblp.py>`_.

    Args:
        in_channels (int or Dict[str, int]): Size of each input sample of every
            node type, or :obj:`-1` to derive the size from the first input(s)
            to the forward method.
        out_channels (int): Size of each output sample.
        metadata (Tuple[List[str], List[Tuple[str, str, str]]]): The metadata
            of the heterogeneous graph, *i.e.* its node and edge types given
            by a list of strings and a list of string triplets, respectively.
            See :meth:`torch_geometric.data.HeteroData.metadata` for more
            information.
        heads (int, optional): Number of multi-head-attentions.
            (default: :obj:`1`)
        group (string, optional): The aggregation scheme to use for grouping
            node embeddings generated by different relations.
            (:obj:`"sum"`, :obj:`"mean"`, :obj:`"min"`, :obj:`"max"`).
            (default: :obj:`"sum"`)
        **kwargs (optional): Additional arguments of
            :class:`torch_geometric.nn.conv.MessagePassing`.
    """
    def __init__(
        self,
        in_channels: Union[int, Dict[str, int]],
        out_channels: int,
        metadata: Metadata,
        heads: int = 1,
        group: str = "sum",
        **kwargs,
    ):
        super().__init__(aggr='add', node_dim=0, **kwargs)

        if not isinstance(in_channels, dict):
            in_channels = {node_type: in_channels for node_type in metadata[0]}

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.group = group

        self.k_lin = torch.nn.ModuleDict()
        self.q_lin = torch.nn.ModuleDict()
        self.v_lin = torch.nn.ModuleDict()
        self.a_lin = torch.nn.ModuleDict()
        self.skip = torch.nn.ParameterDict()
        for node_type, in_channels in self.in_channels.items():
            self.k_lin[node_type] = Linear(in_channels, out_channels, bias=False)
            self.q_lin[node_type] = Linear(in_channels, out_channels, bias=False)
            self.v_lin[node_type] = Linear(in_channels, out_channels, bias=False)
            self.a_lin[node_type] = Linear(out_channels, out_channels, bias=False)
            self.skip[node_type] = Parameter(torch.Tensor(1))

        self.a_rel = torch.nn.ParameterDict()
        self.m_rel = torch.nn.ParameterDict()
        self.p_rel = torch.nn.ParameterDict()
        dim = out_channels // heads
        for edge_type in metadata[1]:
            edge_type = '__'.join(edge_type)
            self.a_rel[edge_type] = Parameter(torch.Tensor(heads, dim, dim))
            self.m_rel[edge_type] = Parameter(torch.Tensor(heads, dim, dim))
            self.p_rel[edge_type] = Parameter(torch.Tensor(heads))

        self.reset_parameters()

    def reset_parameters(self):
        for val in self.k_lin.values():
            init.eye_(val.weight)
        for val in self.q_lin.values():
            init.eye_(val.weight)
        for val in self.v_lin.values():
            init.eye_(val.weight)
        for val in self.a_lin.values():
            init.eye_(val.weight)
        ones(self.skip)
        ones(self.p_rel)
        ones(self.a_rel)
        ones(self.m_rel)


    def forward(
        self,
        x_dict: Dict[NodeType, Tensor],
        edge_index_dict: Union[Dict[EdgeType, Tensor],
                               Dict[EdgeType, SparseTensor]]  # Support both.
    ) -> Dict[NodeType, Optional[Tensor]]:
        r"""
        Args:
            x_dict (Dict[str, Tensor]): A dictionary holding input node
                features  for each individual node type.
            edge_index_dict (Dict[str, Union[Tensor, SparseTensor]]): A
                dictionary holding graph connectivity information for each
                individual edge type, either as a :obj:`torch.LongTensor` of
                shape :obj:`[2, num_edges]` or a
                :obj:`torch_sparse.SparseTensor`.

        :rtype: :obj:`Dict[str, Optional[Tensor]]` - The output node embeddings
            for each node type.
            In case a node type does not receive any message, its output will
            be set to :obj:`None`.
        """

        H, D = self.heads, self.out_channels // self.heads

        k_dict, q_dict, v_dict, out_dict = {}, {}, {}, {}

        # Iterate over node-types:
        for node_type, x in x_dict.items():
            k_dict[node_type] = self.k_lin[node_type](x).view(-1, H, D)
            q_dict[node_type] = self.q_lin[node_type](x).view(-1, H, D)
            v_dict[node_type] = self.v_lin[node_type](x).view(-1, H, D)
            out_dict[node_type] = []

        # Iterate over edge-types:
        for edge_type, edge_index in edge_index_dict.items():
            src_type, _, dst_type = edge_type
            edge_type = '__'.join(edge_type)

            a_rel = self.a_rel[edge_type]
            k = (k_dict[src_type].transpose(0, 1) @ a_rel).transpose(1, 0)

            m_rel = self.m_rel[edge_type]
            v = (v_dict[src_type].transpose(0, 1) @ m_rel).transpose(1, 0)

            # propagate_type: (k: Tensor, q: Tensor, v: Tensor, rel: Tensor)
            out = self.propagate(edge_index, k=k, q=q_dict[dst_type], v=v,
                                 rel=self.p_rel[edge_type], size=None)
            out_dict[dst_type].append(out)

        # Iterate over node-types:
        for node_type, outs in out_dict.items():
            out = group(outs, self.group)

            if out is None:
                out_dict[node_type] = None
                continue

            out = self.a_lin[node_type](F.gelu(out))
            if out.size(-1) == x_dict[node_type].size(-1):
                alpha = self.skip[node_type].sigmoid()
                out = alpha * out + (1 - alpha) * x_dict[node_type]
            out_dict[node_type] = out

        return out_dict


    def message(self, k_j: Tensor, q_i: Tensor, v_j: Tensor, rel: Tensor,
                index: Tensor, ptr: Optional[Tensor],
                size_i: Optional[int]) -> Tensor:

        alpha = (q_i * k_j).sum(dim=-1) * rel
        alpha = alpha / math.sqrt(q_i.size(-1))
        alpha = softmax(alpha, index, ptr, size_i)
        out = v_j * alpha.view(-1, self.heads, 1)
        return out.view(-1, self.out_channels)

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}(-1, {self.out_channels}, '
                f'heads={self.heads})')

In [27]:
hacked_pyg = HackedGTConv(in_channels=channels_dict, out_channels=2, metadata=data.metadata(), heads=2)
hacked_pyg(data.x_dict, data.edge_index_dict)

{'airframe': tensor([[0.6764, 0.2102],
         [0.0000, 0.6764],
         [0.0000, 0.0000],
         [0.0000, 0.0000]], grad_fn=<MmBackward0>),
 'powerplant': tensor([[0.8840, 0.0000],
         [0.0000, 0.6792],
         [0.0000, 0.0000]], grad_fn=<AddBackward0>)}